## Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import pandas as pd
from pathlib import Path
import cvxpy

In [2]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [3]:
%run SVR_Library.ipynb 

## Load data

In [4]:
rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]
# 
Y = boston.target
X = boston.data

In [5]:
#split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 5)

#normalize
scaler = MaxAbsScaler().fit(x_train)
X_train = scaler.transform(x_train); X_test = scaler.transform(x_test)

scaler1 = MaxAbsScaler().fit(y_train.reshape(-1, 1))
Y_train = scaler1.transform(y_train.reshape(-1, 1)).reshape(-1)
Y_test = scaler1.transform(y_test.reshape(-1, 1)).reshape(-1)

## Optimization Function

In [6]:
def opt_bas(C, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_cvxpy(**hyperparameters).fit(X_train, Y_train)

    y_pred = model.predict(X_test)
    
    # rescale
#     y_pred = scaler1.inverse_transform(predict.reshape(-1, 1)).reshape(-1)
    
    # get score
    mape = np.mean(np.abs((Y_test - y_pred)/Y_test))*100
    
    return -mape

## Run Optimization

In [7]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [8]:
# Bounded region of parameter space
pbounds = {'C': (1, 10), 'gamma': (1, 10)}

# Domain reduction function
# bounds_transformer = SequentialDomainReductionTransformer()

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
#                                  bounds_transformer=bounds_transformer
                                )

# keep data
log_path = Path().resolve() / "Logs" / "test1.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(init_points=2, n_iter=400)